In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib notebook
sns.set()

In [2]:
from ngmix.medsreaders import NGMixMEDS, MultiBandNGMixMEDS

mbmeds = MultiBandNGMixMEDS([NGMixMEDS('outputs/meds/y3v02/DES2122+0001/DES2122+0001_r_meds-y3v02.fits.fz')])

In [12]:
mbobs = mbmeds.get_mbobs(0)

In [13]:
for ol in mbobs:
    for o in ol:
        print(np.sum(o.image))

0.0
63505.62357234955
64289.336188316345
64314.4324760437
63797.69507217407


In [ ]:
import fitsio
from meds import MEDS

In [ ]:
m = MEDS('outputs/meds/y3v02/DES2122+0001/DES2122+0001_r_meds-y3v02.fits.fz')

In [ ]:
cat = m.get_cat()

In [ ]:
ind = 0
imind = 3

In [ ]:
cat['orig_col'][ind, imind]

In [ ]:
imdata = m.get_image_info()

In [ ]:
cat['file_id'][ind, imind]

In [ ]:
imdata['image_path'][cat['file_id'][ind, imind]]

In [ ]:
import esutil

In [ ]:
hd = fitsio.read_header(
    imdata['image_path'][cat['file_id'][ind, imind]].strip(), ext=1)
wcs = esutil.wcsutil.WCS({k.lower(): hd[k] for k in hd if k is not None})

In [ ]:
wcs.image2sky(cat['orig_col'][ind, imind]+1, cat['orig_row'][ind, imind]+1)

In [ ]:
cat['ra'][ind], cat['dec'][ind]

In [ ]:
wcs.get_jacobian(cat['orig_col'][ind, imind]+1, cat['orig_row'][ind, imind]+1)

In [ ]:
(cat['dudcol'][ind, imind],
 cat['dudrow'][ind, imind],
 cat['dvdcol'][ind, imind],
 cat['dvdrow'][ind, imind]) 

In [ ]:
im = fitsio.read(imdata['image_path'][cat['file_id'][ind, imind]].strip())

im = im[
    cat['orig_start_row'][ind, imind]:cat['orig_start_row'][ind, imind]+cat['box_size'][ind],
    cat['orig_start_col'][ind, imind]:cat['orig_start_col'][ind, imind]+cat['box_size'][ind]]

im /= im.sum()

In [ ]:
mim = m.get_cutout(ind, imind)
mim /= mim.sum()

In [ ]:
mim.sum()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))
axs[0].imshow(np.arcsinh(im))
axs[0].grid(False)
axs[1].imshow(np.arcsinh(mim))
axs[1].grid(False)
axs[2].imshow((mim - im)/im)
axs[2].grid(False)

In [ ]:
np.allclose(mim, im)

In [ ]:
np.max(np.abs(mim - im))

In [ ]:
import galsim
from matts_misc.simple_des_y3_sims.psf_wrapper import PSFWrapper
from matts_misc.simple_des_y3_sims.des_piff import DES_Piff

In [ ]:
wcs = galsim.FitsWCS(imdata['image_path'][cat['file_id'][ind, imind]].strip(), hdu=1)

In [ ]:
import os

psf_path = os.path.join(
    '/Users/Matt/DESDATA/y3_piff/y3a1-v29',
    os.path.basename(imdata['image_path'][cat['file_id'][ind, imind]].strip()).split('_')[0][3:],
    os.path.basename(imdata['image_path'][cat['file_id'][ind, imind]].strip()).replace('_immasked.fits.fz', '_piff.fits'))

In [ ]:
import piff

In [ ]:
psf = DES_Piff(psf_path)

In [ ]:
mpsf = PSFWrapper(psf, wcs)

In [ ]:
psf_im = mpsf.get_rec(cat['orig_row'][ind, imind], cat['orig_col'][ind, imind])

In [ ]:
mpsf_im = m.get_cutout(ind, imind, type='psf')

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))
axs[0].imshow(np.arcsinh(psf_im))
axs[0].grid(False)
axs[1].imshow(np.arcsinh(mpsf_im))
axs[1].grid(False)
axs[2].imshow(np.arcsinh(mpsf_im - psf_im))
axs[2].grid(False)

In [ ]:
np.allclose(mpsf_im, psf_im)

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(12, 12))

for ind, ax in zip(range(9), axs.ravel()):
    sns.heatmap(m.get_cutout(ind, 1, type='psf'), square=True, ax=ax)